In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
from urllib.parse import parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

In [2]:
from IPython.display import Image
Image(url="https://static.tildacdn.com/tild3361-3935-4166-a132-616437623963/IGF-Thumbnail.png", width=300, height=200)

In [3]:
#
# YouTubeTranscriptApi.get_transcript("bAtxYDZsHW8", languages=['ru', 'en'])

Сессии и спикеры Internet Governance Forum Russia 2010—2021

## Данные о сессиях IGF 2021-2015

In [4]:
def parse_igf_v2(soup, year):

    for report in soup.select("tr > td#left"):

        video_page = report.select(".arrow_presentation a")
        video_url = None
        if len(video_page) > 0:
            video_url = video_page[0]['href']
            if "?p=video" not in video_url:
                video_url = None
            else:
                video_url = parse_qs(video_url)['vid'][0]
                video_url = None if len(video_url) != 11 else video_url
    

        yield {
            'youtube_id': video_url,
            'persons': [p.text for p in report.select("ul li a")],
            'title': report.find("h4").text,
            'summary': report.find("p").text if report.find("p") else None,
            'year': year,
        }

In [5]:
reports = []

In [6]:
sessions_v2 = {
    2021: "https://rigf2021.ru/prog/?p=prog",
    2020: "https://rigf2020.ru/prog/?p=prog",
    2019: "https://rigf2019.ru/prog/?p=prog",
    2018: "https://rigf2018.ru/prog/?p=prog",
    2017: "https://rigf2017.ru/prog/?p=prog",
    2016: "https://rigf2016.ru/prog/?p=prog",
    2015: "https://rigf2015.ru/prog/?p=prog",
}

In [7]:
for (year, link) in sessions_v2.items():
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    for r in parse_igf_v2(soup, year): 
        reports.append(r)

## Данные о сессиях IGF 2014-2011

In [8]:
def parse_igf_v1(soup, year):

    for report in soup.select("table.pro td"):

        if report.find("h3") is None:
            continue

        video_page = report.select(".arrow a")
        video_url = None
        if len(video_page) > 0:
            video_url = video_page[len(video_page)-1]['href']
            if "?p=video" not in video_url:
                video_url = None
            else:
                video_url = parse_qs(video_url)['vid'][0]
                video_url = None if len(video_url) != 11 else video_url

        yield {
            'youtube_id': video_url,
            'persons': [p.text for p in report.select("ul li a")],
            'title': report.find("h3").text,
            'summary': report.find("p").text if report.find("p") else None,
            'year': year,
        }

In [9]:
sessions_v1 = {
    2014: "https://rigf2014.ru/prog/?p=prog",
    2013: "https://rigf2013.ru/prog/?p=prog",
    2012: "https://rigf2012.ru/prog/?p=prog",
    2011: "https://rigf2011.ru/prog/?p=prog",
}

In [10]:
for (year, link) in sessions_v1.items():
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    for r in parse_igf_v1(soup, year):
        reports.append(r)

## Данные о сессии IGF 2010

In [11]:
def parse_igf_v0(soup, year):
    for report in soup.select("tr > td#left"):
        yield {
            'youtube_id': None,
            'persons': [p.text for p in report.select("ul li strong a")],
            'title': report.find("strong a").text,
            'summary': report.find("p").text if report.find("p") else None,
            'year': year,
        }

In [12]:
igfru2010_link = "https://rigf2010.ru/eng/program.php"
page = requests.get(link)
soup = BeautifulSoup(page.text, 'html.parser')
for r in parse_igf_v0(soup, 2010): reports.append(r)

## Транскрипции выступлений

In [13]:
for r in reports:
    try:
        transcript = YouTubeTranscriptApi.get_transcript(r['youtube_id'], languages=['ru'])
        r['video_transcript'] = " ".join([t['text'] for t in transcript])
    except:
        r['video_transcript'] = None

## Все сессии и спикеры

In [14]:
df = pd.DataFrame.from_dict(reports)
df

,youtube_id,persons,title,summary,year,video_transcript
0,248b1oXAvB0,"[Максут Шадаев, Александр Хинштейн, Татьяна Ма...",Церемония открытия,6 Всероссийский молодежный конкурс работ по п...,2021,друзья доброе утро здравствуйте дорогие участн...
1,aJFxUKudzwU,"[Рашид Исмаилов, Мэнди Карвер, Вольфганг Клейн...",Секция 1. Управление интернетом,В июле 2018 года Генеральный секретарь ООН объ...,2021,российский форум по управлению интернетом и се...
2,pdTQahZV5TQ,"[Эльза Ганеева, Андрей Игнатьев, Презентация,...",Секция 2. Новые технологии. Искусственный инте...,В соответствии с «Концепцией развития регулиро...,2021,секция называется новые технологии искусственн...
3,q4R98A4Eh84,"[Милош Вагнер, Наталья Великородняя, Николай Д...",Секция 3. Суверенитет данных,"Суверенитет данных – это идея о том, что данны...",2021,доброе утро уважаемые участники 11 российского...
4,XQZTCXwWIvE,[],Вручение награды Virtuti Interneti и традицион...,В 2010 г. Координационный центр доменов .RU/.Р...,2021,дорогие друзья мы переходим следующему этапу 1...
...,...,...,...,...,...,...
79,None,[],Круглый стол 1. Российская и европейская поз...,Европа - один признанных лидеров в области ра...,2011,NaN
80,None,[],Круглый стол 2. Управление интернетом в Росси...,"Модераторы: Маркус Куммер, Вице-президент ISO...",2011,NaN
81,None,[],Перерыв на кофе,None,2011,NaN
82,None,[],Сценарии будущего развития интернета: взгляд и...,"Большинство экспертов предполагают, что в буд...",2011,NaN


In [18]:
df.to_json('all_data.json')

# Статистика

**Всего сессий**

In [16]:
df.size

504

**Всего персон**

In [17]:
import itertools
all_persons = [r['persons'] for r in reports]
all_persons = set(itertools.chain.from_iterable(all_persons))

len(all_persons)


144

**Докладов с транскрипцией**

In [20]:
len(list(filter(lambda r: r['video_transcript'] is not None, reports)))

KeyError: 'video_transcript'